# MCP + LangGraph 핸즈온 튜토리얼

- 작성자: [테디노트](https://youtube.com/c/teddynote)
- 강의: [패스트캠퍼스 RAG 비법노트](https://fastcampus.co.kr/data_online_teddy)

**참고자료**
- https://modelcontextprotocol.io/introduction
- https://github.com/langchain-ai/langchain-mcp-adapters

## 환경설정

아래 설치 방법을 참고하여 `uv` 를 설치합니다.

**uv 설치 방법**

```bash
# macOS/Linux
curl -LsSf https://astral.sh/uv/install.sh | sh

# Windows (PowerShell)
irm https://astral.sh/uv/install.ps1 | iex
```

**의존성 설치**

```bash
uv pip install -r requirements.txt
```

환경변수를 가져옵니다.

In [65]:
from dotenv import load_dotenv
from langchain_teddynote import logging

load_dotenv(override=True)
logging.langsmith("LangGraph MCP Adapters HandsOn")

LangSmith 추적을 시작합니다.
[프로젝트명]
LangGraph MCP Adapters HandsOn


## MultiServerMCPClient

사전에 `mcp_server_remote.py` 를 실행해둡니다. 터미널을 열고 가상환경이 활성화 되어 있는 상태에서 서버를 실행해 주세요.

> 명령어
```bash
source .venv/bin/activate
python mcp_server_remote.py
```

`async with` 로 일시적인 Session 연결을 생성 후 해제

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_teddynote.messages import ainvoke_graph, astream_graph
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-lite", 
    temperature=0,
    max_tokens=20000,
)

async with MultiServerMCPClient(
    {
        "weather": {
            # 서버의 포트와 일치해야 합니다.(8005번 포트)
            "url": "http://localhost:8005/sse",
            "transport": "sse",
        }
    }
) as client:
    print(client.get_tools())
    agent = create_react_agent(model, client.get_tools())
    answer = await astream_graph(agent, {"messages": "서울의 날씨는 어떠니?"})

[StructuredTool(name='get_weather', description='\n    Get current weather information for the specified location.\n\n    This function simulates a weather service by returning a fixed response.\n    In a production environment, this would connect to a real weather API.\n\n    Args:\n        location (str): The name of the location (city, region, etc.) to get weather for\n\n    Returns:\n        str: A string containing the weather information for the specified location\n    ', args_schema={'properties': {'location': {'title': 'Location', 'type': 'string'}}, 'required': ['location'], 'title': 'get_weatherArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x7201ac63d260>)]

🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
It's always Sunny in 서울
🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - 

다음의 경우에는 session 이 닫혔기 때문에 도구에 접근할 수 없는 것을 확인할 수 있습니다.

In [5]:
await astream_graph(agent, {"messages": "서울의 날씨는 어떠니?"})


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
Error: ClosedResourceError()
 Please fix your mistakes.
🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
죄송합니다. 서울의 날씨 정보를 가져오는 데 문제가 발생했습니다. 다시 시도해 주세요.

{'node': 'agent',
 'content': AIMessageChunk(content='도해 주세요.', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-lite', 'safety_ratings': []}, id='run-e390f857-22b9-411b-b358-91176b89c355', usage_metadata={'input_tokens': -50, 'output_tokens': 27, 'total_tokens': -23, 'input_token_details': {'cache_read': 0}}),
 'metadata': {'langgraph_step': 3,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:8f285b9b-b389-3753-5c74-59e317a89244',
  'checkpoint_ns': 'agent:8f285b9b-b389-3753-5c74-59e317a89244',
  'ls_provider': 'google_genai',
  'ls_model_name': 'models/gemini-2.0-flash-lite',
  'ls_model_type': 'chat',
  'ls_temperature': 0.0,
  'ls_max_tokens': 20000}}

이제 그럼 Async Session 을 유지하며 도구에 접근하는 방식으로 변경해 보겠습니다.

In [7]:
# 1. 클라이언트 생성
client = MultiServerMCPClient(
    {
        "weather": {
            "url": "http://localhost:8005/sse",
            "transport": "sse",
        }
    }
)


# 2. 명시적으로 연결 초기화 (이 부분이 필요함)
# 초기화
await client.__aenter__()

# 이제 도구가 로드됨
print(client.get_tools())  # 도구가 표시됨

[StructuredTool(name='get_weather', description='\n    Get current weather information for the specified location.\n\n    This function simulates a weather service by returning a fixed response.\n    In a production environment, this would connect to a real weather API.\n\n    Args:\n        location (str): The name of the location (city, region, etc.) to get weather for\n\n    Returns:\n        str: A string containing the weather information for the specified location\n    ', args_schema={'properties': {'location': {'title': 'Location', 'type': 'string'}}, 'required': ['location'], 'title': 'get_weatherArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x7201ac2839c0>)]


Error in sse_reader: peer closed connection without sending complete message body (incomplete chunked read)


langgraph 의 에이전트를 생성합니다.

In [8]:
# 에이전트 생성
agent = create_react_agent(model, client.get_tools())

그래프를 실행하여 결과를 확인합니다.

In [9]:
await astream_graph(agent, {"messages": "서울의 날씨는 어떠니?"})


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
It's always Sunny in 서울
🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
서울은 항상 맑습니다.

{'node': 'agent',
 'content': AIMessageChunk(content='은 항상 맑습니다.', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-lite', 'safety_ratings': []}, id='run-11494a9a-c44e-46c9-a4bc-b451f182715d', usage_metadata={'input_tokens': -51, 'output_tokens': 9, 'total_tokens': -42, 'input_token_details': {'cache_read': 0}}),
 'metadata': {'langgraph_step': 3,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:a79d5197-d417-7abc-25e0-1f680eec8156',
  'checkpoint_ns': 'agent:a79d5197-d417-7abc-25e0-1f680eec8156',
  'ls_provider': 'google_genai',
  'ls_model_name': 'models/gemini-2.0-flash-lite',
  'ls_model_type': 'chat',
  'ls_temperature': 0.0,
  'ls_max_tokens': 20000}}

## Stdio 통신 방식

Stdio 통신 방식은 로컬 환경에서 사용하기 위해 사용합니다.

- 통신을 위해 표준 입력/출력 사용

참고: 아래의 python 경로는 수정하세요!

In [30]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langgraph.prebuilt import create_react_agent
from langchain_mcp_adapters.tools import load_mcp_tools
from langchain_google_genai import ChatGoogleGenerativeAI

# Google의 Gemini 모델 초기화
model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-lite", 
    temperature=0,
    max_tokens=20000,
)

# StdIO 서버 파라미터 설정
# - command: Python 인터프리터 경로
# - args: 실행할 MCP 서버 스크립트
server_params = StdioServerParameters(
    command="./mcp_test/bin/python",
    args=["mcp_server_local.py"],
)

# StdIO 클라이언트를 사용하여 서버와 통신
async with stdio_client(server_params) as (read, write):
    # 클라이언트 세션 생성
    async with ClientSession(read, write) as session:
        # 연결 초기화
        await session.initialize()

        # MCP 도구 로드
        tools = await load_mcp_tools(session)
        print(tools)

        # 에이전트 생성
        agent = create_react_agent(model, tools)

        # 에이전트 응답 스트리밍
        await astream_graph(agent, {"messages": "서울의 날씨는 어떠니?"})

[StructuredTool(name='get_weather', description='\n    Get current weather information for the specified location.\n\n    This function simulates a weather service by returning a fixed response.\n    In a production environment, this would connect to a real weather API.\n\n    Args:\n        location (str): The name of the location (city, region, etc.) to get weather for\n\n    Returns:\n        str: A string containing the weather information for the specified location\n    ', args_schema={'properties': {'location': {'title': 'Location', 'type': 'string'}}, 'required': ['location'], 'title': 'get_weatherArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x72018830e7a0>)]

🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
It's always Sunny in 서울
🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - 

## RAG 를 구축한 MCP 서버 사용

- 파일: `mcp_server_rag.py`

사전에 langchain 으로 구축한 `mcp_server_rag.py` 파일을 사용합니다.

stdio 통신 방식으로 도구에 대한 정보를 가져옵니다. 여기서 도구는 `retriever` 도구를 가져오게 되며, 이 도구는 `mcp_server_rag.py` 에서 정의된 도구입니다. 이 파일은 사전에 서버에서 실행되지 **않아도** 됩니다.

In [ ]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_teddynote.messages import astream_graph

# Google의 Gemini 모델 초기화
model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash", 
    temperature=0,
    max_tokens=20000,
)

# RAG 서버를 위한 StdIO 서버 파라미터 설정
server_params = StdioServerParameters(
    command="./mcp_test/bin/python",
    args=["./mcp_server_rag.py"],
)

# StdIO 클라이언트를 사용하여 RAG 서버와 통신
async with stdio_client(server_params) as (read, write):
    
    # 클라이언트 세션 생성
    async with ClientSession(read, write) as session:
        # 연결 초기화
        await session.initialize()

        # MCP 도구 로드 (여기서는 retriever 도구)
        tools = await load_mcp_tools(session)

        # 에이전트 생성 및 실행
        agent = create_react_agent(model, tools)

        # 에이전트 응답 스트리밍
        await astream_graph(
            agent, {"messages": "PDF 파일에 어떤 내용이 있나요?"}
        )

## SSE 방식과 StdIO 방식 혼합 사용

- 파일: `mcp_server_rag.py` 는 StdIO 방식으로 통신
- `langchain-dev-docs` 는 SSE 방식으로 통신

SSE 방식과 StdIO 방식을 혼합하여 사용합니다.

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_anthropic import ChatAnthropic

# Anthropic의 Claude 모델 초기화
model = ChatAnthropic(
    model_name="claude-3-7-sonnet-latest", temperature=0, max_tokens=20000
)

# 1. 다중 서버 MCP 클라이언트 생성
client = MultiServerMCPClient(
    {
        "document-retriever": {
            "command": "./.venv/bin/python",
            # mcp_server_rag.py 파일의 절대 경로로 업데이트해야 합니다
            "args": ["./mcp_server_rag.py"],
            # stdio 방식으로 통신 (표준 입출력 사용)
            "transport": "stdio",
        },
        "langchain-dev-docs": {
            # SSE 서버가 8765 포트에서 실행 중인지 확인하세요
            "url": "http://teddynote.io:8765/sse",
            # SSE(Server-Sent Events) 방식으로 통신
            "transport": "sse",
        },
    }
)


# 2. 비동기 컨텍스트 매니저를 통한 명시적 연결 초기화
await client.__aenter__()

langgraph 의 `create_react_agent` 를 사용하여 에이전트를 생성합니다.

In [10]:
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnableConfig

prompt = (
    "You are a smart agent. "
    "Use `retriever` tool to search on AI related documents and answer questions."
    "Use `langchain-dev-docs` tool to search on langchain / langgraph related documents and answer questions."
    "Answer in Korean."
)
agent = create_react_agent(
    model, client.get_tools(), prompt=prompt, checkpointer=MemorySaver()
)

구축해 놓은 `mcp_server_rag.py` 에서 정의한 `retriever` 도구를 사용하여 검색을 수행합니다.

In [ ]:
config = RunnableConfig(recursion_limit=30, thread_id=1)
await astream_graph(
    agent,
    {
        "messages": "`retriever` 도구를 사용해서 삼성전자가 개발한 생성형 AI 이름을 검색해줘"
    },
    config=config,
)

이번에는 `langchain-dev-docs` 도구를 사용하여 검색을 수행합니다.

In [ ]:
config = RunnableConfig(recursion_limit=30, thread_id=1)
await astream_graph(
    agent,
    {"messages": "langgraph-dev-docs 참고해서 self-rag 의 정의에 대해서 알려줘"},
    config=config,
)

`MemorySaver` 를 사용하여 단기 기억을 유지합니다. 따라서, multi-turn 대화도 가능합니다.

In [ ]:
await astream_graph(
    agent, {"messages": "이전의 내용을 bullet point 로 요약해줘"}, config=config
)

## LangChain 에 통합된 도구 + MCP 도구

여기서는 LangChain 에 통합된 도구를 기존의 MCP 로만 이루어진 도구와 함께 사용이 가능한지 테스트 합니다.

In [ ]:
from langchain_teddynote.tools.tavily import TavilySearch

# Tavily 검색 도구를 초기화 합니다. (news 타입, 최근 3일 내 뉴스)
tavily = TavilySearch(max_results=3, topic="news", days=3)

# 기존의 MCP 도구와 함께 사용합니다.
tools = client.get_tools() + [tavily]

langgraph 의 `create_react_agent` 를 사용하여 에이전트를 생성합니다.

In [15]:
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnableConfig

prompt = "You are a smart agent with various tools. Answer questions in Korean."
agent = create_react_agent(model, tools, prompt=prompt, checkpointer=MemorySaver())

새롭게 추가한 `tavily` 도구를 사용하여 검색을 수행합니다.

In [ ]:
await astream_graph(agent, {"messages": "오늘 뉴스 찾아줘"}, config=config)

`retriever` 도구가 원활하게 작동하는 것을 확인할 수 있습니다.

In [ ]:
await astream_graph(
    agent,
    {
        "messages": "`retriever` 도구를 사용해서 삼성전자가 개발한 생성형 AI 이름을 검색해줘"
    },
    config=config,
)

## Smithery 에서 제공하는 MCP 서버

- 링크: https://smithery.ai/

사용한 도구 목록은 아래와 같습니다.

- Sequential Thinking: https://smithery.ai/server/@smithery-ai/server-sequential-thinking
  - 구조화된 사고 프로세스를 통해 역동적이고 성찰적인 문제 해결을 위한 도구를 제공하는 MCP 서버
- Desktop Commander: https://smithery.ai/server/@wonderwhy-er/desktop-commander
  - 다양한 편집 기능으로 터미널 명령을 실행하고 파일을 관리하세요. 코딩, 셸 및 터미널, 작업 자동화

**참고**

- smithery 에서 제공하는 도구를 JSON 형식으로 가져올때, 아래의 예시처럼 `"transport": "stdio"` 로 꼭 설정해야 합니다.

In [72]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_google_genai import ChatGoogleGenerativeAI

# LLM 모델 초기화
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

# 1. 클라이언트 생성
client = MultiServerMCPClient(
    {
        # "server-sequential-thinking": {
        #     "command": "npx",
        #     "args": [
        #         "-y",
        #         "@smithery/cli@latest",
        #         "run",
        #         "@smithery-ai/server-sequential-thinking",
        #         "--key",
        #         "a94d0233-912b-4752-82e4-78949f447de7"
        #     ],
        #     "transport": "stdio",  # stdio 방식으로 통신을 추가합니다.
        # },
        "desktop-commander": {
            "command": "npx",
            "args": [
                "-y",
                "@smithery/cli@latest",
                "run",
                "@wonderwhy-er/desktop-commander",
                "--key",
                "a94d0233-912b-4752-82e4-78949f447de7"
            ],
            "transport": "stdio",  # stdio 방식으로 통신을 추가합니다.
        },
        # "document-retriever": {
        #     "command": "./.venv/bin/python",
        #     # mcp_server_rag.py 파일의 절대 경로로 업데이트해야 합니다
        #     "args": ["./mcp_server_rag.py"],
        #     # stdio 방식으로 통신 (표준 입출력 사용)
        #     "transport": "stdio",
        # },
        "duckduckgo-mcp-server": {
            "command": "npx",
            "args": [
                "-y",
                "@smithery/cli@latest",
                "run",
                "@nickclyde/duckduckgo-mcp-server",
                "--key",
                "a94d0233-912b-4752-82e4-78949f447de7"
            ],
            "transport": "stdio",  # stdio 방식으로 통신을 추가합니다.
        }
    }
)


# 2. 명시적으로 연결 초기화
await client.__aenter__()

langgraph 의 `create_react_agent` 를 사용하여 에이전트를 생성합니다.

In [73]:
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(recursion_limit=30, thread_id=2)

agent = create_react_agent(model, client.get_tools(), checkpointer=MemorySaver())

`Desktop Commander` 도구를 사용하여 터미널 명령을 실행합니다.

In [74]:
await astream_graph(
    agent,
    {
        "messages": "현재 경로를 포함한 하위 폴더 구조를 tree 로 그려줘. 단, mcp_test 폴더는 제외하고 출력해줘."
    },
    config=config,
)

Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
현재 경로를 알 수 없으므로, tree 구조를 그릴 수 없습니다. 현재 경로를 알려주시거나, 특정 경로를 지정해주시면 해당 경로를 기준으로 tree 구조를 그려드리겠습니다. 또한, mcp_test 폴더를 제외하는 옵션도 적용해드리겠습니다.

{'node': 'agent',
 'content': AIMessageChunk(content=' 그려드리겠습니다. 또한, mcp_test 폴더를 제외하는 옵션도 적용해드리겠습니다.', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run-5f19a43c-d037-4be4-8954-c7e29471e1aa', usage_metadata={'input_tokens': -700, 'output_tokens': 86, 'total_tokens': -614, 'input_token_details': {'cache_read': 0}}),
 'metadata': {'thread_id': 2,
  'langgraph_step': 1,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:883842a7-c389-3fd8-4f37-31172825ba4c',
  'checkpoint_ns': 'agent:883842a7-c389-3fd8-4f37-31172825ba4c',
  'ls_provider': 'google_genai',
  'ls_model_name': 'models/gemini-2.0-flash',
  'ls_model_type': 'chat',
  'ls_temperature': 0.7}}

이번에는 `Sequential Thinking` 도구를 사용하여 비교적 복잡한 작업을 수행할 수 있는지 확인합니다.

In [57]:
await astream_graph(
    agent,
    {
        "messages": (
            "`DuckDuckGo Search Server` 도구를 사용해서 손흥민에 대한 최신 기사 1개를 검색해서 출력해줘."
        )
    },
    config=config,
)


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 


CancelledError: 